## Toy ECCO Working Examples

This table shows three algorithms that are implemented across several languages (Python, Julia, and Fortran). Each algorithm also has been used with an AD method as well. Where possible, we have compared the gradients generated by different AD methods on the same algorithm. In the case of the bulk formulae from Air Sea Fluxes, we have a toy problem that uses Enzyme and Optim (in Julia) to match the output of the formula to some observations. The table roughly summarizes what can be found on the EH24-ToyECCO repository.

Note: the cost function for the optimization problem is just an example, it is not particularly meaningful and does not include any normalization or regularization. 

<img src="Examples_table.png" width="700" />

## Timing Benchmarks

Where relevant we have compared the time it takes to do the automatic differentiation. Julia uses just in time compilation, which means that the first run is slow, and the subsequent runs are fast (comparable to python).

<img src="timing_table.png" width="750" />

## Example L96 Time Series

4-variable Lorenz 96.

<img src="det_series.png" width="700" />

# Notes

## Julia, Enzyme, and Optim

We ran into some difficulties when using Enzyme with Optim.optimization. 

Optim.optimize expects a function, gradient, and one flat vector. The desired form is optimize(f, f_ad!,x0).

In order to pass a function with multiple arguments to optim we needed to use a "closure". For example: x0 -> cost(x0, observations). This prevents us from having to hard code the observations as part of the cost function, and instead to pass them in as an argument. Because they are fixed though, we use the closure to effectively link the obs to the cost function, leaving x as the only input. 

We then need to take the derivative of the closure with Enzyme to get it's adjoint. Unfortunately this only seems to work for simple cases. Enzyme complains with a variety of LLVM compiler errors if the behavior inside the cost function gets more complex. In particular LLVM complains with "LLVM error: failed to perform tail call elimination on a call site marked musttail". 

It seems like the closures are handy but Enzyme does not particularly like them. There is a simple example using Enzyme, Optim and a closure function. There is a slightly more complex example (involving more examples) that does not rely on the closure but hard codes the observations. At some point we would want a middle ground. 
